
#### Problem Statement:
You are tasked with creating a **UserProfileManager** system that manages user profiles. Each profile has the following attributes:
- `username` (must be a non-empty string).
- `email` (must contain "@" and ".").
- `last_login` (a datetime object, nullable).

The system must meet these requirements:

1. **Custom Descriptors for Validation and Storage**:
   - Create a `ValidatedProperty` descriptor that validates data using a provided validation function before assignment.
   - Use `__set_name__` to automatically bind property names.

2. **Strong and Weak References**:
   - Implement a caching mechanism to store recently used profiles. Use weak references to avoid memory leaks.
   - Profiles must be automatically removed from the cache when they are no longer strongly referenced elsewhere.

3. **Property Resolution and Overrides**:
   - Implement a class-level default value for `last_login` that is returned if the value is not explicitly set for an instance.

4. **Testing and Edge Cases**:
   - Write unit tests to verify proper validation, caching, and default handling.

---

#### Deliverables:
1. **Implementation**: A Python file (`user_profile_manager.py`) containing the solution.
2. **Pytest Test Cases**: A file (`test_user_profile_manager.py`) with thorough test cases covering all requirements.
3. **GitHub Actions Workflow**: A file (`.github/workflows/python-tests.yml`) to automate testing on push or pull requests.

---

#### Evaluation Criteria:
- **Correctness**: Does the implementation meet all requirements?
- **Code Quality**: Is the code clean, modular, and well-documented?
- **Testing**: Are all edge cases covered, and do tests pass consistently?
- **Automation**: Is the GitHub Actions workflow correctly implemented?

---

# test_user_profile_manager.py

In [ ]:

import pytest
from user_profile_manager import UserProfileManager

def test_username_validation():
    manager = UserProfileManager()
    with pytest.raises(ValueError):
        manager.username = ""  # Invalid: Empty string
    manager.username = "valid_user"  # Valid
    assert manager.username == "valid_user"

def test_email_validation():
    manager = UserProfileManager()
    with pytest.raises(ValueError):
        manager.email = "invalidemail.com"  # Invalid: Missing '@'
    manager.email = "user@example.com"  # Valid
    assert manager.email == "user@example.com"

def test_last_login_default():
    manager = UserProfileManager()
    assert manager.last_login is None  # Default value

def test_cache_management():
    manager = UserProfileManager()
    manager.username = "cache_test"
    manager.email = "test@example.com"
    uid = id(manager)
    
    # Add to cache and check existence
    UserProfileManager.add_to_cache(manager)
    assert UserProfileManager.get_from_cache(uid) is manager
    
    # Delete strong reference and confirm weak ref removal
    del manager
    assert UserProfileManager.get_from_cache(uid) is None

def test_property_resolution():
    manager = UserProfileManager()
    manager.last_login = None  # Should resolve to default if unset
    assert manager.last_login is None


# python-tests.yml

In [ ]:

name: Python Tests

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    - name: Set up Python
      uses: actions/setup-python@v3
      with:
        python-version: '3.x'
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pytest
    - name: Run tests
      run: pytest
